In [1]:
import os.path

import pandas as pd
import numpy as np

import scipy.signal
import scipy.io
import utils

from pynwb import NWBHDF5IO
from collections import OrderedDict
from tqdm import tqdm


In [2]:
winL = 0.05
frameshift = 0.01
modelOrder = 4
stepSize = 5
path_bids = r'./raw_data'
feat_path = r'./feat'
result_path = r'./res'
# pts = ['sub-%02d'%i for i in range(1,11)]
participants = pd.read_csv(os.path.join(path_bids,'participants.tsv'), delimiter='\t')
pts = participants['participant_id']

In [3]:
def dict4wav(words):
    number_mapping = {
        '0': 'nul',
        '1': 'een',
        '2': 'twee',
        '3': 'drie',
        '4': 'vier',
        '5': 'vijf',
        '6': 'zes',
        '7': 'zeven',
        '8': 'acht',
        '9': 'negen',
        '10': 'tien',
        '11': 'elf',
        '12': 'twaalf',
        '13': 'dertien',
        '14': 'veertien',
        '15': 'vijftien',
        '16': 'zestien',
        '17': 'zeventien',
        '18': 'achttien',
        '19': 'negentien',
        '20': 'twintig',
    }
    words = list(OrderedDict.fromkeys(words))
    new_words = []
    for word in words:
        if word == '':
            continue
        if '`' in word:
            word = word.replace('`','\'')
        if word in number_mapping.keys():
            word = number_mapping[word]
        new_words.append(word) 
    return new_words

In [4]:
os.makedirs(os.path.join(feat_path), exist_ok=True)
for p_id, pt in enumerate(tqdm(pts)):
    os.makedirs(os.path.join(feat_path,f'{pt}'), exist_ok=True)
    #Load data
    io = NWBHDF5IO(os.path.join(path_bids,pt,'ieeg',f'{pt}_task-wordProduction_ieeg.nwb'), 'r')
    nwbfile = io.read()
    #sEEG
    eeg = nwbfile.acquisition['iEEG'].data[:]
    eeg_sr = 1024
    #audio
    audio = nwbfile.acquisition['Audio'].data[:]
    audio_sr = 48000
    #words (markers)
    words = nwbfile.acquisition['Stimulus'].data[:]
    words = np.array(words, dtype=str)
    io.close()
    #channels
    channels = pd.read_csv(os.path.join(path_bids,pt,'ieeg',f'{pt}_task-wordProduction_channels.tsv'), delimiter='\t')
    channels = np.array(channels['name'])

    #Process Audio
    target_SR = 16000
    audio = scipy.signal.decimate(audio,int(audio_sr / target_SR))
    audio_sr = target_SR
    scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
    # scaled[np.abs(scaled)<1000] = 0
    record_words = dict4wav(words)
    # os.makedirs(os.path.join(feat_path,f'{pt}','audio'), exist_ok=True)
    # os.makedirs(os.path.join(path_output,f'{pt}','result'), exist_ok=True)
    audio_length = int(len(scaled)/len(record_words))
    eeg_length = int(len(eeg)/len(record_words))
    word_length = 1
    for i in range(len(record_words)):
        word_info = dict()
        word_info['label'] = record_words[i]
        word_info['audio'] = scaled[i*audio_length:(i+1)*audio_length]
        word_info['eeg'] = eeg[i*eeg_length:(i+1)*eeg_length]
        np.save(os.path.join(feat_path,pt,f'{i}.npy'),word_info)

  0%|          | 0/10 [00:00<?, ?it/s]/data1/byzhao/miniconda3/envs/bci/lib/python3.10/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/data1/byzhao/miniconda3/envs/bci/lib/python3.10/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
/data1/byzhao/miniconda3/envs/bci/lib/python3.10/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)
 10%|█         | 1/10 [00:04<00:37,  4.19s/it]/data1/byzhao/miniconda3/envs/bci/lib/python3.10/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/data1/byzhao/miniconda3/envs